In [17]:
import h5py
import hdf5plugin
import numpy as np
import itertools
import torch

filename = "/Users/michaelzhang/Downloads/ntuple_merged_14.h5"

h5 = h5py.File(filename,'r')

In [29]:
params_2 = ['track_ptrel',     
          'track_erel',     
          'track_phirel',     
          'track_etarel',     
          'track_deltaR',
          'track_drminsv',     
          'track_drsubjet1',     
          'track_drsubjet2',
          'track_dz',     
          'track_dzsig',     
          'track_dxy',     
          'track_dxysig',     
          'track_normchi2',     
          'track_quality',     
          'track_dptdpt',     
          'track_detadeta',     
          'track_dphidphi',     
          'track_dxydxy',     
          'track_dzdz',     
          'track_dxydz',     
          'track_dphidxy',     
          'track_dlambdadz',     
          'trackBTag_EtaRel',     
          'trackBTag_PtRatio',     
          'trackBTag_PParRatio',     
          'trackBTag_Sip2dVal',     
          'trackBTag_Sip2dSig',     
          'trackBTag_Sip3dVal',     
          'trackBTag_Sip3dSig',     
          'trackBTag_JetDistVal'
         ]


labels_qcd = {"label_QCD_b": 53, "label_QCD_bb":51, "label_QCD_c":54,"label_QCD_cc":52,"label_QCD_others": 55}
label_H_bb = 41
num_jets = h5["event_no"].shape[0]
n_particles = h5["track_dxydxy"].shape[-1]

X = []
y = []
edge_indices = []
for jet in range(num_jets):
    if h5["fj_label"][jet] in list(labels_qcd.values()):
        y.append([0,1])
    elif h5["fj_label"][jet] == label_H_bb:
        y.append([1,0])
    else: continue
    
    x = []
    for feature in params_2:
        x.append(h5[feature][jet])
    X.append(x)
    
    # complete graph has n_particles*(n_particles-1)/2 edges, here we double count each edge, so has  n_particles*(n_particles-1) total edges
    pairs = np.stack([[m, n] for (m, n) in itertools.product(range(n_particles), range(n_particles)) if m != n])
    # edge_index = torch.tensor(pairs, dtype=torch.long)
    edge_index = pairs.transpose()
    edge_indices.append(edge_index)
    
    if jet > 10:
        break

In [39]:
torch.tensor(y)

tensor([[0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1]])